In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import pandas as pd
import h2o
from h2o.automl import H2OAutoML, get_leaderboard
h2o.init(ip="localhost")

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "13.0.2" 2020-01-14; Java(TM) SE Runtime Environment (build 13.0.2+8); Java HotSpot(TM) 64-Bit Server VM (build 13.0.2+8, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/p9/p7z03z594sd1wmfvrvgs1_lh0000gn/T/tmp2m6z7fzh
  JVM stdout: /var/folders/p9/p7z03z594sd1wmfvrvgs1_lh0000gn/T/tmp2m6z7fzh/h2o_danielmurphy_started_from_python.out
  JVM stderr: /var/folders/p9/p7z03z594sd1wmfvrvgs1_lh0000gn/T/tmp2m6z7fzh/h2o_danielmurphy_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.1.2
H2O cluster version age:,16 days
H2O cluster name:,H2O_from_python_danielmurphy_oifm0x
H2O cluster total nodes:,1
H2O cluster free memory:,4 Gb
H2O cluster total cores:,16
H2O cluster allowed cores:,16
H2O cluster status:,"accepting new members, healthy"


In [3]:
df = pd.read_csv (r'founder_V0.3_founder.csv')
df.columns = df.columns.str.replace(' ','_')

#Select features for model
y = "Success"
df[y] = df[y].astype('category')

#Set Factors
x_factor = ["Gender", "Headquarters_Location_"]
df[x_factor] = df[x_factor].astype('category')

#Set Numerics
x_numeric=df.columns[pd.Series(df.columns).str.contains('Number').tolist()].tolist() #Get all columns with "Number" in the name
x_numeric.extend(['Founded_Date']) #add any other necessary columns
df[x_numeric] = df[x_numeric].apply(pd.to_numeric)

#get all features together
x = x_numeric + x_factor
x_y = x+[y] # combining predictors and y
df=df[x_y]
data_h2o = h2o.H2OFrame(df)


train, test = data_h2o.split_frame(ratios = [.7], seed = 1234)


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [17]:
aml = H2OAutoML(max_models=10, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [18]:
lb = aml.leaderboard

In [20]:
# Optionally edd extra model information to the leaderboard
lb = get_leaderboard(aml, extra_columns='ALL')
# Print all rows (instead of default 10 rows)
lb.head(rows=lb.nrows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle,training_time_ms,predict_time_per_row_ms
StackedEnsemble_AllModels_AutoML_20200402_191542,0.151825,0.389647,0.151825,0.303991,0.273484,1689,0.127313
StackedEnsemble_BestOfFamily_AutoML_20200402_191542,0.152105,0.390006,0.152105,0.304629,0.273709,782,0.049002
DRF_1_AutoML_20200402_191542,0.154389,0.392924,0.154389,0.300368,0.275615,1023,0.020872
GBM_4_AutoML_20200402_191542,0.155543,0.39439,0.155543,0.308218,0.276569,335,0.014789
GBM_3_AutoML_20200402_191542,0.156284,0.395328,0.156284,0.312175,0.277254,269,0.013645
GBM_2_AutoML_20200402_191542,0.156699,0.395852,0.156699,0.314159,0.277728,303,0.011381
GBM_1_AutoML_20200402_191542,0.157224,0.396515,0.157224,0.315754,0.278265,282,0.010104
XGBoost_1_AutoML_20200402_191542,0.157948,0.397427,0.157948,0.317664,0.278616,2240,0.012234
GBM_5_AutoML_20200402_191542,0.15844,0.398045,0.15844,0.317777,0.279481,266,0.014418
XGBoost_3_AutoML_20200402_191542,0.159309,0.399136,0.159309,0.325794,0.280022,959,0.005033


In [21]:
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20200402_191542

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.09647924394138345
RMSE: 0.3106110814851644
MAE: 0.23883434673195827
RMSLE: 0.21983184675814738
R^2: 0.5953734306380533
Mean Residual Deviance: 0.09647924394138345
Null degrees of freedom: 12904
Residual degrees of freedom: 12897
Null deviance: 3077.070902751219
Residual deviance: 1245.0646430635534
AIC: 6463.398171335158

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.15182459343861296
RMSE: 0.3896467546876439
MAE: 0.30399129962139876
RMSLE: 0.27348410306995635
R^2: 0.36325926823015076
Mean Residual Deviance: 0.15182459343861296
Null degrees of freedom: 12904
Residual degrees of freedom: 12897
Null deviance: 3077.4548876755243
Residual deviance: 1959.2963783253003
AIC: 12314.498941036545


In [22]:
preds = aml.predict(test_data= test)

preds.head(15)
test.concat(preds)

stackedensemble prediction progress: |████████████████████████████████████| 100%


Number_of_News_Articles,Number_of_Founded_Organizations,Number_of_Portfolio_Companies,Number_of_Investments_x,Number_of_Partner_Investments,Number_of_Lead_Investments_x,Number_of_Exits_x,Number_of_Events_x,Number_of_Founders,Founded_Date,Gender,Headquarters_Location_,Success,predict
14445,7,11,21,0,9,6,2,4,2003,1,California,1,0.899756
128,1,6,7,0,0,1,5,2,2006,1,California,1,0.987523
3669,1,73,101,0,3,17,19,2,1999,1,California,1,0.950903
90,1,0,0,0,0,0,0,2,1999,1,China,1,1.01592
48,1,3,3,0,0,2,0,3,2014,1,California,1,0.834022
451,1,2,3,0,0,1,2,1,2014,1,New York,1,0.541971
1330,3,1,1,0,0,0,12,1,2016,0,New York,0,0.277839
436,1,0,0,0,0,0,0,1,2000,1,Massachusetts,1,0.987908
234,1,3,3,0,0,0,0,5,2013,1,Nevada,0,0.320035
0,3,0,0,0,0,0,0,5,2013,1,Nevada,0,0.0514043
